Ονοματεπώνυμο: Αποστολία Χρυσοβαλάντου Σκέντζου

Αριθμός μητρώου: el20054

In [ ]:
! pip install transformers datasets
! pip install evaluate
! pip install sentence-transformers
! pip install transformers[torch]
! pip install accelerate -U
!pip install tabulate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl

### Προσοχή

Μη διαγράψετε τα # insert your code here σχόλια, καθώς βοηθούν στη διόρθωση. Συμπληρώστε τον κώδικά σας μετά από τα σχόλια αυτά.

# Μέρος Α: Fine-tune a pretrained model

Τα γλωσσικά μοντέλα αποτελούνται από δύο στάδια εκπαίδευσης:
1. Pre-training σε μεγάλα unlabelled datasets. Το pre-training είναι υπολογιστικά πολύ ακριβό και γι αυτό στην πράξη δε το χρησιμοποιούμε όταν θέλουμε να τρέξουμε ένα μοντέλο σε ένα καινούργιο dataset. Μπορούμε να σκεφτούμε το pre-training ως τη διαδικασία εκμάθησης γλωσσικών κανόνων κι εννοιών, οι οποίες στη συνέχεια μπορούν να χρησιμοποιηθούν για διάφορους σκοπούς.

2. Fine-tuning σε μικρότερα labelled datasets. Το fine-tuning πρακτικά εκμεταλλεύεται τις ιδιότητες του transfer learning προκειμένου να μεταφέρουμε τη 'γνώση' που έχει αποθηκευθεί στο γλωσσικό μοντέλο κατά τη διάρκεια του pre-training σε συγκεκριμένα task. Κάθε task εξυπηρετείται μέσω στοχευμένων datasets. Για παράδειγμα, κάποια datasets αναφέρονται στην ταξινόμηση κειμένων σε κατηγιορίες (text classification), άλλα datasets περιέχουν ερωτήσεις οι οποίες πρέπει να απαντηθούν (question answering) κι άλλα πολλά.

Κάποια κλασικά tasks της επεξεργασίας φυσικής γλώσσας είναι τα ακόλουθα:
- Text classification
- Question answering
- Natural language inference
- Fill mask
- Semantic similarity

Περισσότερες πληροφορίες μπορείτε να βρείτε στον ακόλουθο σύνδεσμο στο domain Natural Language Processing: https://huggingface.co/models

Στο πρώτο κομμάτι της παρούσας εργαστηριακής άσκησης, θα χρησιμοποιήσουμε το pre-training fine-tuning σενάριο για να ταξινομήσουμε reviews.

## Pipelines

Με τη χρήση του text-classification pipeline μπορούμε να τρέξουμε γλωσσικά μοντέλα που αφορούν tasks ταξινόμησης. Το natural language inference (NLI) task αποτελεί ένα task ταξινόμησης, αφού το σχετικό μοντέλο (εν προκειμένω το roberta-large-mnli) καλείται να ταξινομήσει ένα κείμενο σε μία από τις 3 κατηγορίες [entailment/neutral/contradiction].

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "roberta-large-mnli")
classifier("A soccer game with multiple males playing. Some men are playing a sport.")
## [{'label': 'ENTAILMENT', 'score': 0.98}]
```

Ένα άλλο task ταξινόμησης αφορά την αξιολόγηση του κατά πόσο ένα κείμενο είναι γραμματικά ορθό (acceptable) ή όχι (unacceptable):

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "textattack/distilbert-base-uncased-CoLA")
classifier("I will walk to home when I went through the bus.")
##  [{'label': 'unacceptable', 'score': 0.95}]
```

## Σύνολο δεδομένων Yelp polarity

Κατεβάζουμε το [Yelp Polarity](https://huggingface.co/datasets/yelp_polarity) dataset το οποίο περιέχει reviews που εκφράζουν συναισθήματα πελατών για εστιατόρια. Τα reviews αυτά χωρίζονται σε κατηγορίες, και ο σκοπός μας είναι να κατηγοριοποιήσουμε νέα reviews στις σωστές κατηγορίες.

In [ ]:
from datasets import load_dataset

# insert your code here
dataset = load_dataset("yelp_polarity")
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for yelp_polarity contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/yelp_polarity
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.war

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 560000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 38000
    })
})


Επειδή το σύνολο δεδομένων του Yelp Polarity περιέχει πολλά δείγματα, προκειμένου να επιταχύνουμε τη διαδικασία του fine-tuning συστήνουμε να διατηρήσετε 300 δείγματα από το train set και 300 δείγματα από το test set.

Ελέγξτε τον αριθμό κατηγοριών που υπάρχουν συνολικά στο train και το test set και διατηρήστε ισορροπημένο αριθμό δειγμάτων ανά κατηγορία για τα σύνολα αυτά κατά την επιλογή των 300 δειγμάτων.

In [ ]:
#insert your code here

import random
from collections import defaultdict

# Defining number of samples
number_of_samples = 300

# Create dictionaries
train_samples_per_category = defaultdict(list)
test_samples_per_category = defaultdict(list)

# Iterate through the dataset and distribute samples to the dictionaries
for example in dataset['train']:
    label = example['label']
    train_samples_per_category[label].append(example)

for example in dataset['test']:
    label = example['label']
    test_samples_per_category[label].append(example)


selected_train_samples = []
selected_test_samples = []

num_samples_per_category_train = {}
num_samples_per_category_test = {}

desired_samples_per_category = number_of_samples // len(train_samples_per_category)

# Select samples from each category
for label, samples in train_samples_per_category.items():
    selected_train_samples.extend(random.sample(samples, desired_samples_per_category))

for label, samples in test_samples_per_category.items():
    selected_test_samples.extend(random.sample(samples, desired_samples_per_category))

# Shuffle the selected samples
random.shuffle(selected_train_samples)
random.shuffle(selected_test_samples)

# Print the number of selected samples per category
print("Number of selected samples per category in the training set:")
for label, samples in train_samples_per_category.items():
    print(f"Category {label}: {desired_samples_per_category}")

print("\nNumber of selected samples per category in the test set:")
for label, samples in test_samples_per_category.items():
    print(f"Category {label}: {desired_samples_per_category}")

# Print the total number of selected samples for both training and test sets
print("\nTotal number of selected samples:")
print(f"Training set: {len(selected_train_samples)}")
print(f"Test set: {len(selected_test_samples)}")


Number of selected samples per category in the training set:
Category 0: 150
Category 1: 150

Number of selected samples per category in the test set:
Category 1: 150
Category 0: 150

Total number of selected samples:
Training set: 300
Test set: 300


In [ ]:
from datasets import Dataset

# coverting into dataset format
train_dataset = Dataset.from_dict({"text": [sample["text"] for sample in selected_train_samples],
                                   "label": [sample["label"] for sample in selected_train_samples]})

test_dataset = Dataset.from_dict({"text": [sample["text"] for sample in selected_test_samples],
                                  "label": [sample["label"] for sample in selected_test_samples]})


In [ ]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 300
})

# Language Models

Η προεπεξεργασία των κειμένων προηγείται της εισόδου τους στα γλωσσικά μοντέλα. Η διαδικασία αυτή επιτελείται μέσω των Tokenizers, τα οποία μετατρέπουν τα tokens εισόδου σε κατάλληλα IDs του λεξιλογίου προεκπαίδευσης, κι έτσι μετατρέπουν το κείμενο σε μορφή που μπορεί να επεξεργαστεί κάποιο μοντέλο Transformer. Η βιβλιοθήκη Huggingface προσφέρει εύκολες και high-level υλοποιήσεις tokenization, τις οποίες συστήνουμε να ακολουθήσετε στη συνέχεια.

Συγκεκριμένα, αρχικοποιούμε τη διαδικασία του tokenization με χρήση του AutoTokenizer. Επιλέγοντας τη μέθοδο from_pretrained λαμβάνουμε έναν tokenizer που αποκρίνεται στην αρχιτεκτονική του μοντέλου που επιθυμούμε να χρησιμοποιήσουμε, παρέχοντας συμβατό tokenization.

Περισσότερες πληροφορίες για το AutoTokenization μπορείτε να βρείτε εδώ:
https://huggingface.co/docs/transformers/model_doc/auto

Αναφορικά με το μοντέλο BERT το οποίο διδαχθήκατε στο εργαστήριο, μπορείτε να δείτε τη διαδικασία [του tokenization και της αρχικοποίησης του μοντέλου](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizer):

```
from transformers import AutoTokenizer, BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
```

Στα πλαίσια της άσκησης καλείστε να επιτελέσετε την παραπάνω διαδικασία με *κάποιο άλλο μοντέλο της επιλογής σας από το Huggingface* που να υποστηρίζει τον AutoTokenizer. Το pre-trained μοντέλο που θα επιλέξετε θα πρέπει να διαθέτει υλοποίηση με sequence classification head (κατ αναλογία της μεθόδου BertForSequenceClassification).

Στο επόμενο κελί, φορτώστε το επιλεχθέν μοντέλο με τον αντίστοιχο tokenizer.

(Αγνοήστε πιθανά warnings της μορφής Some weights of the model checkpoint at xxx were not used when initializing...)

In [ ]:
# insert your code here

from transformers import AutoTokenizer, AutoModelForSequenceClassification
# we are choosing electra as our model
tokenizer = AutoTokenizer.from_pretrained("google/electra-small-discriminator")
model = AutoModelForSequenceClassification.from_pretrained("google/electra-small-discriminator")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Σας παρέχουμε τη συνάρτηση που πραγματοποιεί το tokenization καλώντας τον tokenizer που επιλέξατε. Εφαρμόστε το τόσο στο train, όσο και στο test set.

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# insert your code here

# tokenize training set
train_dataset = train_dataset.map(tokenize_function, batched=True)

# tokenize test set
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Τυπώνοντας το train ή το test set, θα δείτε δύο επιπλέον πεδία 'input_ids' και 'attention_mask'. Βεβαιωθείτε ότι υπάρχουν, άρα και το tokenization έχει επιτευχθεί.

In [ ]:
train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 300
})

## Χρήση του PyTorch Trainer για fine-tuning

Η κλάση [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) έχει βελτιστοποιηθεί από τους δημιουργούς του Huggingface παρέχοντας πολλές διευκολύνσεις και λιγότερη 'χεράτη' δουλειά. Προτείνουμε να τη χρησιμοποιήσετε ως εναλλακτική του να γράψετε το δικό σας training loop.
Καθώς η Trainer δεν τεστάρει αυτόματα την επίδοση του εκάστοτε μοντέλου κατά την εκπαίδευση, παρέχουμε κατάλληλη συνάρτηση προκειμένου να αποτιμάται το accuracy του μοντέλου σε κάθε εποχή.

In [ ]:
import numpy as np
import evaluate
import torch
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Η κλάση [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) περιέχει όλες τις υπερπαραμέτρους με τις οποίες μπορείτε να πειραματιστείτε κατά τη διαδικασία fine-tuning.


Καλείστε να πειραματιστείτε με διαφορετικές υπερπαραμέτρους όπως το learning rate, batch size κλπ, καθώς επίσης και να ορίσετε optimizer και scheduler για το fine-tuning. Προτείνουμε να εκτελέσετε fine-tuning για μικρό αριθμό εποχών (άλλωστε το μοντέλο είναι ήδη προεκπαιδευμένο).

1. Θα μας δώσετε σε markdown ένα πινακάκι με διαφορετικές υπερπαραμέτρους που δοκιμάσατε και το accuracy που πετύχατε στην τελευταία εποχή.

2. Βάσει των πειραματισμών, πώς επηρεάζουν διαφορετικές υπερπαράμετροι όπως το learning rate και το batch size το fine-tuning του μοντέλου που επιλέξατε? Σχολιάστε και αναλύστε.

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

Then fine-tune your model by calling [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [ ]:
trained_model=trainer.train()

Θα μεταβάλλουμε τις τιμές για μία παράμετρο την φορά και θα βγάζουμε έτσι συμπεράσματα για το πόσο σημαντικό ρόλο παίζει η συγκεκριμένη παράμετρος για την διαδικασία του fine-tuning, αλλά και ποια είναι η βέλτιστη τιμή που μπορούμε να επιλέξουμε.

Αρχικά, θα πειραματιστούμε με τις τιμές της υπερπαραμέτρου **learning rate**, η οποία καθορίζει το ρυθμό με τον οποίο ανανεώνονται τα βάρη μέσω του back propagation στη διαδικασία της εκπαίδευσης.

In [ ]:
from sklearn.model_selection import ParameterGrid
from transformers import TrainingArguments, Trainer, AdamW, Adafactor

# Define hyperparameter values
hyperparameters_grid = {
    'learning_rate': [5e-5, 3e-5, 2e-5],
    'per_device_train_batch_size': [16],
    'weight_decay': [0.001],
    'adam_epsilon': [1e-8],
    'warmup_steps': [100],
}

# Generate combinations of hyperparameters
hyperparameter_combinations = list(ParameterGrid(hyperparameters_grid))

# Initialize a dictionary to store results
results = []

# Iterate over hyperparameter combinations
for i, params in enumerate(hyperparameter_combinations):
    print(f"Training model with hyperparameters: {params}")

    # Initialize tokenizer with max_seq_length
    tokenizer = AutoTokenizer.from_pretrained("google/electra-small-discriminator", max_length=128)

    # Initialize model with current hyperparameters
    model = AutoModelForSequenceClassification.from_pretrained("google/electra-small-discriminator")

    # Initialize Trainer with current hyperparameters
    trainer = Trainer(
        model=model,
        args=TrainingArguments(
            **params,
            num_train_epochs=3,
            logging_dir=f'./logs/{i}',
            logging_steps=100,
            evaluation_strategy="steps",
            eval_steps=100,
            save_steps=500,
            output_dir=f'./results/{i}',
            overwrite_output_dir=True,
        ),
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    evaluation_result = trainer.evaluate()
    accuracy = evaluation_result['eval_accuracy']

    # Store the results
    results.append({**params, 'accuracy': accuracy})

# Display results in a table
import pandas as pd
results_df = pd.DataFrame(results)
print(results_df)



Training model with hyperparameters: {'adam_epsilon': 1e-08, 'learning_rate': 5e-05, 'per_device_train_batch_size': 16, 'warmup_steps': 100, 'weight_decay': 0.001}


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss


Training model with hyperparameters: {'adam_epsilon': 1e-08, 'learning_rate': 3e-05, 'per_device_train_batch_size': 16, 'warmup_steps': 100, 'weight_decay': 0.001}


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss


Training model with hyperparameters: {'adam_epsilon': 1e-08, 'learning_rate': 2e-05, 'per_device_train_batch_size': 16, 'warmup_steps': 100, 'weight_decay': 0.001}


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss


   adam_epsilon  learning_rate  per_device_train_batch_size  warmup_steps  \
0  1.000000e-08        0.00005                           16           100   
1  1.000000e-08        0.00003                           16           100   
2  1.000000e-08        0.00002                           16           100   

   weight_decay  accuracy  
0         0.001  0.770000  
1         0.001  0.633333  
2         0.001  0.573333  


In [ ]:
from tabulate import tabulate

table = tabulate(results_df, headers='keys', tablefmt='pretty')
print(table)


+---+--------------+---------------+-----------------------------+--------------+--------------+--------------------+
|   | adam_epsilon | learning_rate | per_device_train_batch_size | warmup_steps | weight_decay |      accuracy      |
+---+--------------+---------------+-----------------------------+--------------+--------------+--------------------+
| 0 |    1e-08     |     5e-05     |            16.0             |    100.0     |    0.001     |        0.77        |
| 1 |    1e-08     |     3e-05     |            16.0             |    100.0     |    0.001     | 0.6333333333333333 |
| 2 |    1e-08     |     2e-05     |            16.0             |    100.0     |    0.001     | 0.5733333333333334 |
+---+--------------+---------------+-----------------------------+--------------+--------------+--------------------+


Παίρνουμε τώρα την βέλτιστη τιμή που βρήκαμε στα παραπάνω test για το learning_rate. Θα δοκιμάσουμε τώρα διαφορετικές τιμές για την υπερπαράμετρο **batch_size**. Όπως είναι γνωστό αυτή η παράμετρος σχετίζεται με τον αριθμό των samples που θα χρησιμοποιηθούν για κάθε iteration.

In [ ]:
from sklearn.model_selection import ParameterGrid
from transformers import TrainingArguments, Trainer, AdamW, Adafactor
from tabulate import tabulate

model = AutoModelForSequenceClassification.from_pretrained("google/electra-small-discriminator")

# Define hyperparameters to test
hyperparameters_grid = {
    'learning_rate': [5e-5],
    'per_device_train_batch_size': [16, 32, 64],
    'weight_decay': [0.001],
    'adam_epsilon': [1e-8],
    'warmup_steps': [100],
}

# Generate combinations of hyperparameters
hyperparameter_combinations = list(ParameterGrid(hyperparameters_grid))

# Initialize a dictionary to store results
results = []

# Iterate over hyperparameter combinations
for i, params in enumerate(hyperparameter_combinations):
    print(f"Training model with hyperparameters: {params}")

    # Initialize tokenizer with max_seq_length
    tokenizer = AutoTokenizer.from_pretrained("google/electra-small-discriminator", max_length=128)

    # Initialize model with current hyperparameters
    model = AutoModelForSequenceClassification.from_pretrained("google/electra-small-discriminator")

    # Initialize Trainer with current hyperparameters
    trainer = Trainer(
        model=model,
        args=TrainingArguments(
            **params,
            num_train_epochs=3,
            logging_dir=f'content/drive/My Drive/DLlab2/logs/{i}',
            logging_steps=100,
            evaluation_strategy="steps",
            eval_steps=100,
            save_steps=500,
            output_dir=f'content/drive/My Drive/DLlab2/results/{i}',
            overwrite_output_dir=True,
        ),
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    evaluation_result = trainer.evaluate()
    accuracy = evaluation_result['eval_accuracy']

    # Store the results
    results.append({**params, 'accuracy': accuracy})

# Display results in a table
import pandas as pd
results_df = pd.DataFrame(results)

# Convert DataFrame to a formatted table
table = tabulate(results_df, headers='keys', tablefmt='pretty')

# Print the formatted table
print(table)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training model with hyperparameters: {'adam_epsilon': 1e-08, 'learning_rate': 5e-05, 'per_device_train_batch_size': 16, 'warmup_steps': 100, 'weight_decay': 0.001}


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


Training model with hyperparameters: {'adam_epsilon': 1e-08, 'learning_rate': 5e-05, 'per_device_train_batch_size': 32, 'warmup_steps': 100, 'weight_decay': 0.001}


...

Step,Training Loss,Validation Loss


Training model with hyperparameters: {'adam_epsilon': 1e-08, 'learning_rate': 5e-05, 'per_device_train_batch_size': 64, 'warmup_steps': 100, 'weight_decay': 0.001}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


+---+--------------+---------------+-----------------------------+--------------+--------------+--------------------+
|   | adam_epsilon | learning_rate | per_device_train_batch_size | warmup_steps | weight_decay |      accuracy      |
+---+--------------+---------------+-----------------------------+--------------+--------------+--------------------+
| 0 |    1e-08     |     5e-05     |            16.0             |    100.0     |    0.001     | 0.7833333333333333 |
| 1 |    1e-08     |     5e-05     |            32.0             |    100.0     |    0.001     | 0.5666666666666667 |
| 2 |    1e-08     |     5e-05     |            64.0             |    100.0     |    0.001     |        0.6         |
+---+--------------+---------------+-----------------------------+--------------+--------------+--------------------+


Παίρνουμε τώρα τις βέλτιστες τιμές των παρμέτρων που βρήκαμε στα παραπάνω test. Θα δοκιμάσουμε τώρα διαφορετικές τιμές για την υπερπαράμετρο **weight_decay**, η οποία εξαναγκάζει ομαλοποίηση στα δεδομένα μας, ¨τιμωρώντας" τα μεγάλα βάρη για να πετύχουμε καλύτερη γενίκευση στο μοντέλο.

In [ ]:
from sklearn.model_selection import ParameterGrid
from transformers import TrainingArguments, Trainer, AdamW, Adafactor
from tabulate import tabulate

model = AutoModelForSequenceClassification.from_pretrained("google/electra-small-discriminator")

# Define hyperparameters to test
hyperparameters_grid = {
    'learning_rate': [5e-5],
    'per_device_train_batch_size': [16],
    'weight_decay': [0.00001, 0.0001, 0.001, 0.01, 0.1],
    'adam_epsilon': [1e-8],
    'warmup_steps': [100],
}

# Generate combinations of hyperparameters
hyperparameter_combinations = list(ParameterGrid(hyperparameters_grid))

# Initialize a dictionary to store results
results = []

# Iterate over hyperparameter combinations
for i, params in enumerate(hyperparameter_combinations):
    print(f"Training model with hyperparameters: {params}")

    # Initialize tokenizer with max_seq_length
    tokenizer = AutoTokenizer.from_pretrained("google/electra-small-discriminator", max_length=128)

    # Initialize model with current hyperparameters
    model = AutoModelForSequenceClassification.from_pretrained("google/electra-small-discriminator")

    # Initialize Trainer with current hyperparameters
    trainer = Trainer(
        model=model,
        args=TrainingArguments(
            **params,
            num_train_epochs=3,
            logging_dir=f'content/drive/My Drive/DLlab2/logs/{i}',
            logging_steps=100,
            evaluation_strategy="steps",
            eval_steps=100,
            save_steps=500,
            output_dir=f'content/drive/My Drive/DLlab2/results/{i}',
            overwrite_output_dir=True,
        ),
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    evaluation_result = trainer.evaluate()
    accuracy = evaluation_result['eval_accuracy']

    # Store the results
    results.append({**params, 'accuracy': accuracy})

# Display results in a table
import pandas as pd
results_df = pd.DataFrame(results)

# Convert DataFrame to a formatted table
table = tabulate(results_df, headers='keys', tablefmt='pretty')

# Print the formatted table
print(table)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training model with hyperparameters: {'adam_epsilon': 1e-08, 'learning_rate': 5e-05, 'per_device_train_batch_size': 16, 'warmup_steps': 100, 'weight_decay': 1e-05}


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


Training model with hyperparameters: {'adam_epsilon': 1e-08, 'learning_rate': 5e-05, 'per_device_train_batch_size': 16, 'warmup_steps': 100, 'weight_decay': 0.0001}


...

Step,Training Loss,Validation Loss


Training model with hyperparameters: {'adam_epsilon': 1e-08, 'learning_rate': 5e-05, 'per_device_train_batch_size': 16, 'warmup_steps': 100, 'weight_decay': 0.1}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


+---+--------------+---------------+-----------------------------+--------------+--------------+--------------------+
|   | adam_epsilon | learning_rate | per_device_train_batch_size | warmup_steps | weight_decay |      accuracy      |
+---+--------------+---------------+-----------------------------+--------------+--------------+--------------------+
| 0 |    1e-08     |     5e-05     |            16.0             |    100.0     |    1e-05     | 0.7233333333333334 |
| 1 |    1e-08     |     5e-05     |            16.0             |    100.0     |    0.0001    | 0.7266666666666667 |
| 2 |    1e-08     |     5e-05     |            16.0             |    100.0     |    0.001     | 0.7266666666666667 |
| 3 |    1e-08     |     5e-05     |            16.0             |    100.0     |     0.01     | 0.7266666666666667 |
| 4 |    1e-08     |     5e-05     |            16.0             |    100.0     |     0.1      | 0.7266666666666667 |
+---+--------------+---------------+--------------------

Παίρνουμε τώρα την βέλτιστες τιμές των παραπάνω υπερπαραμέτρων και θα δοκιμάσουμε διαφορετικές τιμές για την υπερπαράμετρο **adam_epsilon**.

In [ ]:
from sklearn.model_selection import ParameterGrid
from transformers import TrainingArguments, Trainer, AdamW, Adafactor
from tabulate import tabulate

model = AutoModelForSequenceClassification.from_pretrained("google/electra-small-discriminator")

# Define hyperparameters to test
hyperparameters_grid = {
    'learning_rate': [5e-5],
    'per_device_train_batch_size': [16],
    'weight_decay': [0.0001],
    'adam_epsilon': [1e-9, 1e-8, 1e-10],
    'warmup_steps': [100],
}

# Generate combinations of hyperparameters
hyperparameter_combinations = list(ParameterGrid(hyperparameters_grid))

# Initialize a dictionary to store results
results = []

# Iterate over hyperparameter combinations
for i, params in enumerate(hyperparameter_combinations):
    print(f"Training model with hyperparameters: {params}")

    # Initialize tokenizer with max_seq_length
    tokenizer = AutoTokenizer.from_pretrained("google/electra-small-discriminator", max_length=128)

    # Initialize model with current hyperparameters
    model = AutoModelForSequenceClassification.from_pretrained("google/electra-small-discriminator")

    # Initialize Trainer with current hyperparameters
    trainer = Trainer(
        model=model,
        args=TrainingArguments(
            **params,
            num_train_epochs=3,
            logging_dir=f'content/drive/My Drive/DLlab2/logs/{i}',
            logging_steps=100,
            evaluation_strategy="steps",
            eval_steps=100,
            save_steps=500,
            output_dir=f'content/drive/My Drive/DLlab2/results/{i}',
            overwrite_output_dir=True,
        ),
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    evaluation_result = trainer.evaluate()
    accuracy = evaluation_result['eval_accuracy']

    # Store the results
    results.append({**params, 'accuracy': accuracy})

# Display results in a table
import pandas as pd
results_df = pd.DataFrame(results)

# Convert DataFrame to a formatted table
table = tabulate(results_df, headers='keys', tablefmt='pretty')

# Print the formatted table
print(table)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training model with hyperparameters: {'adam_epsilon': 1e-09, 'learning_rate': 5e-05, 'per_device_train_batch_size': 16, 'warmup_steps': 100, 'weight_decay': 0.0001}


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss


...

Step,Training Loss,Validation Loss


Training model with hyperparameters: {'adam_epsilon': 1e-10, 'learning_rate': 5e-05, 'per_device_train_batch_size': 16, 'warmup_steps': 100, 'weight_decay': 0.0001}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


+---+--------------+---------------+-----------------------------+--------------+--------------+--------------------+
|   | adam_epsilon | learning_rate | per_device_train_batch_size | warmup_steps | weight_decay |      accuracy      |
+---+--------------+---------------+-----------------------------+--------------+--------------+--------------------+
| 0 |    1e-09     |     5e-05     |            16.0             |    100.0     |    0.0001    |        0.76        |
| 1 |    1e-08     |     5e-05     |            16.0             |    100.0     |    0.0001    | 0.7266666666666667 |
| 2 |    1e-10     |     5e-05     |            16.0             |    100.0     |    0.0001    | 0.7266666666666667 |
+---+--------------+---------------+-----------------------------+--------------+--------------+--------------------+


Τέλος θα πειραματιστούμε με διαφορετικές τιμές για την υπερπαράμετρο **warmup_steps**

In [ ]:
from sklearn.model_selection import ParameterGrid
from transformers import TrainingArguments, Trainer, AdamW, Adafactor
from tabulate import tabulate

model = AutoModelForSequenceClassification.from_pretrained("google/electra-small-discriminator")

# Define hyperparameters to test
hyperparameters_grid = {
    'learning_rate': [5e-5],
    'per_device_train_batch_size': [16],
    'weight_decay': [0.0001],
    'adam_epsilon': [1e-9],
    'warmup_steps': [0.01, 0.05, 0.1],
}

# Generate combinations of hyperparameters
hyperparameter_combinations = list(ParameterGrid(hyperparameters_grid))

# Initialize a dictionary to store results
results = []

# Iterate over hyperparameter combinations
for i, params in enumerate(hyperparameter_combinations):
    print(f"Training model with hyperparameters: {params}")

    # Initialize tokenizer with max_seq_length
    tokenizer = AutoTokenizer.from_pretrained("google/electra-small-discriminator", max_length=128)

    # Initialize model with current hyperparameters
    model = AutoModelForSequenceClassification.from_pretrained("google/electra-small-discriminator")

    # Initialize Trainer with current hyperparameters
    trainer = Trainer(
        model=model,
        args=TrainingArguments(
            **params,
            num_train_epochs=3,
            logging_dir=f'content/drive/My Drive/DLlab2/logs/{i}',
            logging_steps=100,
            evaluation_strategy="steps",
            eval_steps=100,
            save_steps=500,
            output_dir=f'content/drive/My Drive/DLlab2/results/{i}',
            overwrite_output_dir=True,
        ),
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    evaluation_result = trainer.evaluate()
    accuracy = evaluation_result['eval_accuracy']

    # Store the results
    results.append({**params, 'accuracy': accuracy})

# Display results in a table
import pandas as pd
results_df = pd.DataFrame(results)

# Convert DataFrame to a formatted table
table = tabulate(results_df, headers='keys', tablefmt='pretty')

# Print the formatted table
print(table)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training model with hyperparameters: {'adam_epsilon': 1e-09, 'learning_rate': 5e-05, 'per_device_train_batch_size': 16, 'warmup_steps': 0.01, 'weight_decay': 0.0001}


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss


Training model with hyperparameters: {'adam_epsilon': 1e-09, 'learning_rate': 5e-05, 'per_device_train_batch_size': 16, 'warmup_steps': 0.05, 'weight_decay': 0.0001}


...

Step,Training Loss,Validation Loss


Training model with hyperparameters: {'adam_epsilon': 1e-09, 'learning_rate': 5e-05, 'per_device_train_batch_size': 16, 'warmup_steps': 0.1, 'weight_decay': 0.0001}


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss


+---+--------------+---------------+-----------------------------+--------------+--------------+--------------------+
|   | adam_epsilon | learning_rate | per_device_train_batch_size | warmup_steps | weight_decay |      accuracy      |
+---+--------------+---------------+-----------------------------+--------------+--------------+--------------------+
| 0 |    1e-09     |     5e-05     |            16.0             |     0.01     |    0.0001    | 0.7633333333333333 |
| 1 |    1e-09     |     5e-05     |            16.0             |     0.05     |    0.0001    | 0.7366666666666667 |
| 2 |    1e-09     |     5e-05     |            16.0             |     0.1      |    0.0001    | 0.7366666666666667 |
+---+--------------+---------------+-----------------------------+--------------+--------------+--------------------+


In [ ]:
print(table)

+---+--------------+---------------+-----------------------------+--------------+--------------+--------------------+
|   | adam_epsilon | learning_rate | per_device_train_batch_size | warmup_steps | weight_decay |      accuracy      |
+---+--------------+---------------+-----------------------------+--------------+--------------+--------------------+
| 0 |    1e-09     |     5e-05     |            16.0             |     0.01     |    0.0001    | 0.7633333333333333 |
| 1 |    1e-09     |     5e-05     |            16.0             |     0.05     |    0.0001    | 0.7366666666666667 |
| 2 |    1e-09     |     5e-05     |            16.0             |     0.1      |    0.0001    | 0.7366666666666667 |
+---+--------------+---------------+-----------------------------+--------------+--------------+--------------------+


**Παραρητήσεις:**

Παρατηρούμε πως οι διαφορετικές υπερπαράμετροι επηρεάζουν διαφορετικά το accuracy του μοντέλου μας. Συγκεκριμένα, όπως είναι λογικό, όσο βελτιστοποιούμε περισσότερες υπερπαραμέτρους τόσο αυξάνεται το accuracy του fine-tuned μοντέλου μας.  Ακόμη, σημειώνουμε πως οι δύο πρώτες υπερπαράμετροι, το **learning_rate** και **batch_size**, είναι αυτά που έχουν την μεγαλύτερη επίδραση στο accuracy, καθώς σε αυτές παρατηρούμε τις μεγαλύετρες διακυμάνσεις. Οι βέλτιστες τιμες παρατηρούνται για το μεγαλύτερο learning rate που δοκιμάσαμε και το μικρότερο batch_size.

# Μέρος Β: Χρήση fine-tuned μοντέλων σε νέα tasks

Στο κομμάτι αυτό της εργασίας δε χρειάζεται να πραγματοποιήσετε εκπαίδευση σε γλωσσικά μοντέλα. Αντιθέτως, θα εκμεταλλευτούμε τις δυνατότητες του transfer learning για να αντιμετωπίσουμε πιο πολύπλοκα γλωσσικά task, ανάγοντάς τα σε κλασικά task όπως είναι το text classification, natural language inference, question answering και άλλα.

Για παράδειγμα, fine-tuned μοντέλα για [text classification](https://huggingface.co/tasks/text-classification) εξυπηρετούν tasks όπως:

- Είναι δύο προτάσεις η μία παράφραση της άλλης? [Paraphrase/No Paraphrase]
- Συνεπάγεται η πρόταση Χ την πρόταση Υ? [Entail/Neutral/Contradict]
- Είναι η δοθείσα πρόταση γραμματικά ορθή? [Acceptable/Unacceptable]

## B1. Piqa dataset

Το [Piqa dataset](https://huggingface.co/datasets/piqa) περιλαμβάνει προτάσεις οι οποίες ελέγχουν το βαθμό στον οποίο τα language models έχουν κοινή γνώση (commonsense). Συγκεκριμένα, αποτελείται από προτάσεις και πιθανά endings, τα οποία απαιτούν commonsense γνώση για να συμπληρωθούν.

Για παράδειγμα, έχοντας την πρόταση "When boiling butter, when it's ready, you can" υπάρχουν δύο υποψήφια endings:
- "Pour it onto a plate"
- "Pour it into a jar"

Ένας άνθρωπος μπορεί να συμπεράνει ότι η δεύτερη πρόταση αποτελεί ένα πιο κατάλληλο ending, αφού το λιωμένο βούτυρο είναι υγρό, άρα το βάζο είναι ένα καταλληλότερο δοχείο σε σχέση με το πιάτο.

Για λόγους επιτάχυνσης επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το Piqa.

In [ ]:
# # insert your code here (load dataset)
from datasets import load_dataset
from random import sample as random_sample

# loading 100 random samples from PIQA dataset
piqa = load_dataset('piqa')
total_samples = len(piqa['train'])
random_indices = random_sample(range(total_samples), 100)
sample_data = piqa['train'].select(random_indices)

Μπορούμε να θεωρήσουμε το παραπάνω σενάριο σαν ένα πρόβλημα πολλαπλής επιλογής, όπου υπάρχουν δύο πιθανές εναλλακτικές για το ending της πρότασης. Συνεπώς, αξιοποιώντας σχετικά μοντέλα μπορούμε να επιλύσουμε την επιλογή του ending δοθείσας της πρότασης.

Καλείστε λοιπόν να καταγράψετε το accuracy πρόβλεψης endings για κάθε πρόταση με χρήση γλωσσικών μοντέλων. Για λόγους σύγκρισης χρησιμοποιήστε τουλάχιστον 5 κατάλληλα μοντέλα.

# Predicting with models for Multiple Choice tasks

In [ ]:
# insert your code here (models)

# initiating the 5 models i need

model_names_mc = [
    'albert/albert-base-v2',
    'google-bert/bert-base-uncased',
    'FacebookAI/roberta-base',
    'xlnet/xlnet-base-cased',
    'google/bigbird-roberta-base'
]

In [ ]:
# insert your code here (function for ending prediction)

from transformers import AutoTokenizer, AlbertForMultipleChoice,  BertForMultipleChoice, RobertaForMultipleChoice, XLNetForMultipleChoice, BigBirdForMultipleChoice
import torch


# Define the prediction function
def make_predictions(model_name, samples):
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    if(model_name == 'albert/albert-base-v2'):
      model = AlbertForMultipleChoice.from_pretrained("albert/albert-base-v2")
    elif(model_name == 'google-bert/bert-base-uncased'):
      model = BertForMultipleChoice.from_pretrained("google-bert/bert-base-uncased")
    elif(model_name == 'FacebookAI/roberta-base'):
      model = RobertaForMultipleChoice.from_pretrained("FacebookAI/roberta-base")
    elif(model_name == 'xlnet/xlnet-base-cased'):
      model = XLNetForMultipleChoice.from_pretrained("xlnet/xlnet-base-cased")
    elif(model_name == 'google/bigbird-roberta-base'):
      model = BigBirdForMultipleChoice.from_pretrained("google/bigbird-roberta-base")

    correct_predictions = 0


    # Preprocess the samples
    for example in samples:
        goal = example['goal']
        choice0 = example['sol1']
        choice1 = example['sol2']
        labels = torch.tensor(0).unsqueeze(0)

        encoding = tokenizer([goal, goal], [choice0, choice1], return_tensors="pt", padding=True)
        outputs = model(**{k: v.unsqueeze(0) for k, v in encoding.items()}, labels=labels)

        loss = outputs.loss
        logits = outputs.logits
        predicted_class = logits.argmax().item()

        if predicted_class == example['label']:
            correct_predictions += 1

    accuracy = correct_predictions / len(samples)
    print(f"\nAccuracy: {accuracy * 100:.2f}%")
    return accuracy


In [ ]:
## making the predictions for each model

import pandas as pd

# list to display the accuracy scores for each model
results = []

for model_name in model_names_mc:
    print(f"\nEvaluating model: {model_name}")
    accuracy = make_predictions(model_name, sample_data)
    results.append({'Model Name': model_name, 'Accuracy': accuracy})

results_df = pd.DataFrame(results)


Evaluating model: albert/albert-base-v2


Some weights of AlbertForMultipleChoice were not initialized from the model checkpoint at albert/albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Accuracy: 44.00%

Evaluating model: google-bert/bert-base-uncased


Some weights of BertForMultipleChoice were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Accuracy: 53.00%

Evaluating model: FacebookAI/roberta-base


...


Accuracy: 47.00%

Evaluating model: xlnet/xlnet-base-cased


Some weights of XLNetForMultipleChoice were not initialized from the model checkpoint at xlnet/xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Accuracy: 46.00%

Evaluating model: google/bigbird-roberta-base


Some weights of BigBirdForMultipleChoice were not initialized from the model checkpoint at google/bigbird-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Attention type 'block_sparse' is not possible if sequence_length: 17 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...



Accuracy: 48.00%


In [ ]:
print(results_df)

                      Model Name  Accuracy
0          albert/albert-base-v2      0.44
1  google-bert/bert-base-uncased      0.53
2        FacebookAI/roberta-base      0.47
3         xlnet/xlnet-base-cased      0.46
4    google/bigbird-roberta-base      0.48


# Predicting with models for Sequence Classification tasks

In [ ]:
# insert your code here (models)

# initiating the 5 models i need
model_names_sc = [
    'distilbert-base-uncased-finetuned-sst-2-english',
    'roberta-base-openai-detector',
    'albert-base-v2',
    'roberta-large-mnli',
    'facebook/bart-large-mnli'
]

In [ ]:
# insert your code here (function for ending prediction)

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Define the prediction function
def make_predictions(model_name, samples):
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # Preprocess the samples
    def preprocess(example):
        goal = example['goal']
        sol1 = example['sol1']
        sol2 = example['sol2']

        input_1 = tokenizer(goal + " " + sol1, truncation=True, padding='max_length', max_length=128, return_tensors='pt')
        input_2 = tokenizer(goal + " " + sol2, truncation=True, padding='max_length', max_length=128, return_tensors='pt')

        return input_1, input_2

    preprocessed_data = [preprocess(example) for example in samples]

    # Prediction function
    def predict(input_1, input_2):
        with torch.no_grad():
            outputs_1 = model(**input_1)
            outputs_2 = model(**input_2)

        logits_1 = outputs_1.logits
        logits_2 = outputs_2.logits

        # Compare logits to choose the best ending
        if logits_1.argmax() > logits_2.argmax():
            return 0  # Choose sol1
        else:
            return 1  # Choose sol2

    # Make predictions and calculate accuracy
    correct_predictions = 0

    for i, (input_1, input_2) in enumerate(preprocessed_data):
        prediction = predict(input_1, input_2)
        correct_label = samples[i]['label']
        #print(f"Sample {i+1} Prediction: {prediction}, Correct Label: {correct_label}")

        if prediction == correct_label:
            correct_predictions += 1

    accuracy = correct_predictions / len(samples)
    print(f"\nAccuracy: {accuracy * 100:.2f}%")
    return accuracy


In [ ]:
## making the predictions for each model

import pandas as pd

# list to display the accuracy scores for each model
results = []

for model_name in model_names_sc:
    print(f"\nEvaluating model: {model_name}")
    accuracy = make_predictions(model_name, sample_data)
    results.append({'Model Name': model_name, 'Accuracy': accuracy})

results_df = pd.DataFrame(results)


Evaluating model: distilbert-base-uncased-finetuned-sst-2-english


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Accuracy: 51.00%

Evaluating model: roberta-base-openai-detector


Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Accuracy: 47.00%

Evaluating model: albert-base-v2


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Accuracy: 48.00%

Evaluating model: roberta-large-mnli


Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Accuracy: 53.00%

Evaluating model: facebook/bart-large-mnli

Accuracy: 50.00%


In [ ]:
print(results_df)

                                        Model Name  Accuracy
0  distilbert-base-uncased-finetuned-sst-2-english      0.51
1                     roberta-base-openai-detector      0.47
2                                   albert-base-v2      0.48
3                               roberta-large-mnli      0.53
4                         facebook/bart-large-mnli      0.50


## B2. Truthful QA

### Sentence Transformers

Οι sentence transformers χρησιμοποιούνται για να δημιουργήσουν embeddings προτάσεων, δηλαδή διανυσματικές αναπαραστάσεις των προτάσεων αυτών σε ένα διανυσματικό χώρο. Χάρη στον τρόπο που έχουν προεκπαιδευτεί, έχουν την ικανότητα να τοποθετούν νοηματικά όμοιες προτάσεις κοντά τη μία στην άλλη, ενώ απομακρύνουν νοηματικά μακρινές προτάσεις. Έτσι, χάρη στις αναπαραστάσεις που λαμβάνουμε από τα sentence embeddings μπορούμε να αξιολογήσουμε σε τι βαθμό δύο προτάσεις είναι κοντά ή μακριά νοηματικά.

Η σύγκριση των διανυσματικών αναπαραστάσεων μπορεί να γίνει κλασικά μέσω μεθόδων όπως το consine similarity, με μεγαλύτερες τιμές μεταξύ διανυσμάτων να σηματοδοτούν πιο όμοια διανύσματα, άρα και πιο όμοιες προτάσεις. Δίνουμε για το λόγο αυτό μια συνάρτηση υπολογισμού του cosine similarity.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

Για παράδειγμα, εκτελέστε το ακόλουθο κελί, το οποίο δίνει μια τιμή ομοιότητας στο διάστημα [0, 1] για δύο προτάσεις ("This is an example sentence", "Each sentence is converted"). Μπορείτε ακόμα να δοκιμάσετε να εκτελέσετε το ακόλουθο κελί για διαφορετικές προτάσεις της επιλογής σας, που μπορεί να είναι όμοιες ή πολύ διαφορετικές μεταξύ τους, και να παρατηρήσετε τις αλλαγές τιμών του cosine similarity.

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)

get_cosine_similarity(embeddings[0], embeddings[1])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

0.40488452

Για τη συνέχεια της άσκησης, καλείστε να επιλέξετε τουλάχιστον 6 διαφορετικά [μοντέλα για semantic similarity](https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads) από τους sentence transformers

### Μπορούν τα question answering μοντέλα να διαχωρίσουν αληθείς και ψευδείς προτάσεις?

Αυτό το ερώτημα θα το απαντήσουμε στο παρόν κομμάτι της άσκησης. Για το λόγο αυτό, φορτώνουμε το dataset [Truthful QA generation](https://huggingface.co/datasets/truthful_qa/viewer/generation/validation), το οποίο περιέχει τις εξής επιλογές:

- best answer
- correct answer
- incorrect answer

Πολλές φορές το best answer και το correct answer είναι ίδια ή έστω πολύ κοντινά νοηματικά. Σε αυτό το σημείο είναι που θα αξιοποιήσουμε το semantic similarity για να αξιολογήσουμε την ομοιότητα αυτή.

Φιλτράρουμε το dataset ώστε να περιέχονται 100 δείγματα συνολικά για λόγους επιτάχυνσης, εκ των οποίων καθένα θα πρέπει να περιέχει τουλάχιστον 2 correct answer. Θεωρούμε έτσι 4 υποψήφιες επιλογές:

1η επιλογή: best answer  
2η επιλογή: 1ο correct answer  
3η επιλογή: 2ο correct answer  
4η επιλογή: incorrect answer  

Οι επιλογές αυτές μαζί με την ερώτηση δίνονται σε ένα μοντέλο πολλαπλής επιλογής σαν αυτά που χρησιμοποιήθηκαν στο ερώτημα Β1. Μπορείτε να θεωρήσετε τα ίδια μοντέλα και να τα επεκτείνετε σε 4 υποψήφιες απαντήσεις.  

Το semantic similarity θα επηρεάσει το τι θεωρούμε βέλτιστα σωστή απάντηση, άρα και το accuracy. Συγκεκριμένα, θα λάβουμε διανυσματικές αναπαραστάσεις για το best answer και τα 2 correct answer που έχουν δοθεί ως υποψήφιες επιλογές μέσω κάποιου semantic similarity μοντέλου. Σε περίπτωση λοιπόν που το μοντέλο πολλαπλής επιλογής προβλέψει ένα εκ των correct answer, και η ομοιότητά τους σε σχέση με το best model ξεπερνάει ένα προεπιλεγμένο κατώφλι ομοιότητας, η απάντηση θεωρείται βέλτιστα σωστή. Θέτουμε λοιπόν κατώφλι ομοιότητας το 0.95.

Για παράδειγμα, έστω ότι το μοντέλο πολλαπλής επιλογής μεταξύ των υποψηφίων [best, 1st correct, 2nd correct, incorrect] επιλέγει το δεύτερο στοιχείο, δηλαδή το 1st correct, και δεδομένου ότι το cosine similarity μεταξύ των embeddings του best και του 1st correct είναι > 0.95, τότε θεωρούμε ότι η απάντηση είναι βέλτιστα σωστή, και συνυπολογίζεται θετικά στο accuracy.

Καλείστε λοιπόν να γράψετε μια συνάρτηση που να υπολογίζει το accuracy εύρεσης βέλτιστα σωστών απαντήσεων ανάμεσα στις υποψήφιες απαντήσεις, εξετάζοντας τουλάχιστον 6 semantic similarity μοντέλα καθώς επίσης και τα μοντέλα που επιλέξατε στο ερώτημα Β1.


In [ ]:
# insert your code here (load dataset)

from datasets import load_dataset
import random

# Load the TruthfulQA dataset for generation tasks
dataset = load_dataset("truthful_qa", "generation")

split = "validation"
all_samples = dataset[split]
samples_with_two_correct_answers = []

for sample in all_samples:
    correct_answers = sample['correct_answers']
    num_correct_answers = sum(answer in correct_answers for answer in sample['correct_answers'])
    if num_correct_answers >= 2:
        samples_with_two_correct_answers.append(sample)

selected_samples = random.sample(samples_with_two_correct_answers, 100)

print("Total number of selected samples:", len(selected_samples))

Total number of selected samples: 100


In [ ]:
# insert your code here (load models for semantic similarity and QA)

# initializing the 6 models for semantic similarity
model_names_ss = [
    "sentence-transformers/all-MiniLM-L6-v2",
    "sentence-transformers/all-mpnet-base-v2",
    "sentence-transformers/all-MiniLM-L12-v2",
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    "sentence-transformers/bert-base-nli-mean-tokens",
    "sentence-transformers/multi-qa-MiniLM-L6-cos-v1",
]

In [ ]:
# insert your code here (function for optimal correct answers & semantic similarity)

from transformers import AutoTokenizer, AlbertForMultipleChoice,  BertForMultipleChoice, RobertaForMultipleChoice, XLNetForMultipleChoice, BigBirdForMultipleChoice
from sentence_transformers import SentenceTransformer
import torch


# Define the prediction function
def make_predictions_sim(model_name, sem_sim_model_name, selected_samples):
  # Load tokenizer and model for multiple choice
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  if(model_name == 'albert/albert-base-v2'):
    model = AlbertForMultipleChoice.from_pretrained("albert/albert-base-v2")
  elif(model_name == 'google-bert/bert-base-uncased'):
    model = BertForMultipleChoice.from_pretrained("google-bert/bert-base-uncased")
  elif(model_name == 'FacebookAI/roberta-base'):
    model = RobertaForMultipleChoice.from_pretrained("FacebookAI/roberta-base")
  elif(model_name == 'xlnet/xlnet-base-cased'):
    model = XLNetForMultipleChoice.from_pretrained("xlnet/xlnet-base-cased")
  elif(model_name == 'google/bigbird-roberta-base'):
    model = BigBirdForMultipleChoice.from_pretrained("google/bigbird-roberta-base")

  model_emb = SentenceTransformer(sem_sim_model_name)

  correct_predictions = 0

  # Loop through each selected sample
  for sample in selected_samples:
      #print(sample)
      question = sample['question']
      best_answer = sample['best_answer']
      correct_answer1 = sample['correct_answers'][0]
      correct_answer2 = sample['correct_answers'][1]
      incorrect_answer = sample['incorrect_answers'][0]
      labels = torch.tensor(0).unsqueeze(0)

      inputs = tokenizer([[question, best_answer], [question, correct_answer1], [question, correct_answer2], [question, incorrect_answer]], return_tensors="pt", padding=True)

      best_answer_embedding = model_emb.encode(best_answer)
      correct_answer1_embedding = model_emb.encode(correct_answer1)
      correct_answer2_embedding = model_emb.encode(correct_answer2)
      incorrect_answer_embedding = model_emb.encode(incorrect_answer)

      # Perform prediction
      outputs = model(**{k: v.unsqueeze(0) for k, v in inputs.items()}, labels=labels)
      logits = outputs.logits
      predicted_class = logits.argmax().item()

      #print(predicted_class)

      if predicted_class == 0:
        prediction_embedding = best_answer_embedding
      elif predicted_class == 1:
        prediction_embedding = correct_answer1_embedding
      elif predicted_class == 2:
        prediction_embedding = correct_answer2_embedding
      else:
        prediction_embedding = incorrect_answer_embedding

      sim =  get_cosine_similarity(best_answer_embedding, prediction_embedding)

      #print("Similarity is: ", sim)

      if(sim>0.95):
        correct_predictions += 1

  accuracy = correct_predictions / len(selected_samples)
  print(f"\nAccuracy: {accuracy * 100:.2f}%")
  return accuracy




In [ ]:
_## making the predictions for each model

import pandas as pd

results_dict = {}

for ss_model_name in model_names_ss:
    results = []
    for mc_model_name in model_names_mc:
        print(f"\nEvaluating multiple-choice model: {mc_model_name} with single-sentence model: {ss_model_name}")
        accuracy = make_predictions_sim(mc_model_name, ss_model_name, selected_samples)
        results.append({'Multiple Choice Model Name': mc_model_name, 'Accuracy': accuracy})

    # Convert results to a DataFrame and store in the dictionary
    results_df = pd.DataFrame(results)
    results_dict[ss_model_name] = results_df



Evaluating multiple-choice model: albert/albert-base-v2 with single-sentence model: sentence-transformers/all-MiniLM-L6-v2


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of AlbertForMultipleChoice were not initialized from the model checkpoint at albert/albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Accuracy: 28.00%

Evaluating multiple-choice model: google-bert/bert-base-uncased with single-sentence model: sentence-transformers/all-MiniLM-L6-v2


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Accuracy: 36.00%

Evaluating multiple-choice model: FacebookAI/roberta-base with single-sentence model: sentence-transformers/all-MiniLM-L6-v2


...


Accuracy: 32.00%

Evaluating multiple-choice model: xlnet/xlnet-base-cased with single-sentence model: sentence-transformers/multi-qa-MiniLM-L6-cos-v1


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of XLNetForMultipleChoice were not initialized from the model checkpoint at xlnet/xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Accuracy: 32.00%

Evaluating multiple-choice model: google/bigbird-roberta-base with single-sentence model: sentence-transformers/multi-qa-MiniLM-L6-cos-v1


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BigBirdForMultipleChoice were not initialized from the model checkpoint at google/bigbird-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Attention type 'block_sparse' is not possible if sequence_length: 40 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...



Accuracy: 37.00%


In [ ]:
# Print all the results together
for ss_model_name, df in results_dict.items():
    print(f"\nResults for Semantic Similarity Model: {ss_model_name}")
    print(df)


Results for Semantic Similarity Model: sentence-transformers/all-MiniLM-L6-v2
      Multiple Choice Model Name  Accuracy
0          albert/albert-base-v2      0.28
1  google-bert/bert-base-uncased      0.36
2        FacebookAI/roberta-base      0.40
3         xlnet/xlnet-base-cased      0.42
4    google/bigbird-roberta-base      0.36

Results for Semantic Similarity Model: sentence-transformers/all-mpnet-base-v2
      Multiple Choice Model Name  Accuracy
0          albert/albert-base-v2      0.33
1  google-bert/bert-base-uncased      0.29
2        FacebookAI/roberta-base      0.28
3         xlnet/xlnet-base-cased      0.51
4    google/bigbird-roberta-base      0.28

Results for Semantic Similarity Model: sentence-transformers/all-MiniLM-L12-v2
      Multiple Choice Model Name  Accuracy
0          albert/albert-base-v2      0.40
1  google-bert/bert-base-uncased      0.32
2        FacebookAI/roberta-base      0.35
3         xlnet/xlnet-base-cased      0.26
4    google/bigbird-roberta-ba

## Β3. Winogrande dataset

Το [Winogrande dataset](https://huggingface.co/datasets/winogrande) αποτελείται από προτάσεις που μία λέξη τους έχει αφαιρεθεί και δίνονται δύο πιθανές επιλογές συμπλήρωσης του κενού. Για παράδειγμα, δοθείσας της πρότασης "John moved the couch from the garage to the backyard to create space. The _ is small.", υπάρχουν δύο πιθανές εναλλακτικές:

- "garage"
- "backyard"

Η δυσκολία της συμπλήρωσης έγκειται στο ότι και οι δύο λέξεις αναφέρονται στην πρόταση, οπότε το μοντέλο θα πρέπει να διαθέτει υψηλές δυνατότητες κατανόησης γλώσσας προκειμένου να επιλέξει μια νοηματικά σωστή συμπλήρωση.

Για λόγους επιτάχυνσης, επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το training set του Winogrande.


In [ ]:
# insert your code here (load dataset)

from datasets import load_dataset
import random

# Load Winogrande dataset
dataset = load_dataset('winogrande', 'winogrande_xl')
total_samples = len(dataset['train'])

# selecting 100 random samples
from random import sample as random_sample
random_indices = random_sample(range(total_samples), 10)
sample_data = dataset['train'].select(random_indices)

# making a transformation in tha dataset, instead od option1, option2 -> options[option1,option2]
def transform_example(example):
    sentence1 = example['sentence']
    options = [example['option1'], example['option2']]

    transformed_example = {'sentence': sentence1, 'options': options, 'answer': example['answer']}
    return transformed_example

# Apply the transformation to the sample data
transformed_data = [transform_example(example) for example in sample_data]

Με κατάλληλο [μετασχηματισμό](https://huggingface.co/DeepPavlov/roberta-large-winogrande) της παραπάνω εισόδου (πρόταση με κενό και δύο επιλογές συμπλήρωσης), καλείστε να καταγράψετε το accuracy σχετικών μοντέλων που επιλύουν το πρόβλημα, συγκρίνοντας το predicted label με το πραγματικό label (1: πρώτη επιλογή, 2: δεύτερη επιλογή). Ουσιαστικά θα πρέπει να αναγάγετε το παραπάνω πρόβλημα σε κάποιο πιο κλασικό πρόβλημα της επεξεργασίας φυσικής γλώσσας.

Δοκιμάστε τουλάχιστον 3 κατάλληλα μοντέλα από το Huggingface για να προσεγγίσετε το πρόβλημα του Winogrande. Προτείνουμε τη χρήση pipelines για τη διευκόλυνσή σας.

In [ ]:
# insert your code here (load models)

Τα μοντέλα που επιλέξαμε για το pipeline είναι ήδη fine-tuned σε αυτό το dataset

In [ ]:
# insert your code here (create pipelines)
pipes = [
    pipeline("zero-shot-classification", model="DeepPavlov/roberta-large-winogrande"),
    pipeline("zero-shot-classification", model="sileod/deberta-v3-large-tasksource-nli"),
    pipeline("zero-shot-classification", model="sileod/deberta-v3-base-tasksource-nli")
]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


config.json:   0%|          | 0.00/18.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [ ]:
# insert your code here (function for predicting best fill)

def make_predictions(my_pipe, dataset):

  correct_predictions = 0

  for sample in transformed_data:
      result = my_pipe(sample['sentence'], sample['options'])

      # Convert the answer to an integer
      answer_index = int(sample['answer']) - 1

      if result['labels'][0] == sample['options'][answer_index]:
          correct_predictions += 1

      #print(sample)
      #print("Prediction is: ", result['labels'][0])

  # Calculate accuracy
  accuracy = correct_predictions / len(transformed_data)
  print("Accuracy is: ", accuracy)

  return accuracy

In [ ]:
# make predictions for all the models

import pandas as pd

# list to display the accuracy scores for each model
results = []

for pipe in pipes:
    print(f"\nEvaluating pipe: {pipe}")
    accuracy = make_predictions(pipe, transformed_data)
    results.append({'Pipe': pipe, 'Accuracy': accuracy})

results_df = pd.DataFrame(results)


Evaluating pipe: <transformers.pipelines.zero_shot_classification.ZeroShotClassificationPipeline object at 0x7dc35d94e770>
Accuracy is:  0.9

Evaluating pipe: <transformers.pipelines.zero_shot_classification.ZeroShotClassificationPipeline object at 0x7dbc3026bd60>
Accuracy is:  0.9

Evaluating pipe: <transformers.pipelines.zero_shot_classification.ZeroShotClassificationPipeline object at 0x7dc314abc760>
Accuracy is:  0.7


In [ ]:
print(results_df)

                                                Pipe  Accuracy
0  <transformers.pipelines.zero_shot_classificati...       0.9
1  <transformers.pipelines.zero_shot_classificati...       0.9
2  <transformers.pipelines.zero_shot_classificati...       0.7
